In [1]:
!pip install datasets
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
from datasets import load_dataset
data=load_dataset('lewtun/github-issues',split='train')
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [3]:
data['is_pull_request'][:5]

[True, True, True, True, True]

In [4]:
data['comments'][1]

['There is a speed up in Windows machines:\r\n- From `13m 52s` to `11m 10s`\r\n\r\nIn Linux machines, some workers crash with error message:\r\n```\r\nOSError: [Errno 12] Cannot allocate memory\r\n```',
 'There is also a speed up in Linux machines:\r\n- From `7m 30s` to `5m 32s`']

In [5]:
data=data.filter(lambda x: (x['is_pull_request']==False and len(x['comments'])>0))

Filter:   0%|          | 0/3019 [00:00<?, ? examples/s]

In [6]:
data

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [7]:
data['comments'][0]

['Cool, I think we can do both :)',
 '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).']

In [8]:
all_columns=data.column_names
columns_to_keep=['title','body','comments','html_url']
columns_to_remove=set(columns_to_keep).symmetric_difference(all_columns)
data=data.remove_columns(columns_to_remove)
data

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [9]:
data.set_format(type='pandas')
df=data[:]
df_comments=df.explode('comments',ignore_index=True)
df_comments

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...
4,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Well it can cause issue with anyone that updat...,## Describe the bug\r\nAfter upgrading to data...
...,...,...,...,...
2959,https://github.com/huggingface/datasets/issues/2,Issue to read a local dataset,My first bug report ❤️\r\nLooking into this ri...,"Hello,\r\n\r\nAs proposed by @thomwolf, I open..."
2960,https://github.com/huggingface/datasets/issues/2,Issue to read a local dataset,"Ok, there are some news, most good than bad :l...","Hello,\r\n\r\nAs proposed by @thomwolf, I open..."
2961,https://github.com/huggingface/datasets/issues/2,Issue to read a local dataset,"Ok great, so as discussed today, let's:\r\n- h...","Hello,\r\n\r\nAs proposed by @thomwolf, I open..."
2962,https://github.com/huggingface/datasets/issues/2,Issue to read a local dataset,Good plan!\r\n\r\nYes I do use `builder_kwargs...,"Hello,\r\n\r\nAs proposed by @thomwolf, I open..."


In [10]:
from datasets import Dataset
data_comments=Dataset.from_pandas(df_comments)
data_comments

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [11]:
data_comments['comments'][0]

'Cool, I think we can do both :)'

In [12]:
data_comments=data_comments.map(
    lambda x: {'comment_length':len(x['comments'].split())}
)
data_comments

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2964
})

In [14]:
data_comments=data_comments.filter(lambda x: x['comment_length']>15)
data_comments

Filter:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2175
})

In [15]:
def text_concat(sample):
  return {
      'text': sample['title']+' \n '+sample['body']+' \n '+sample['comments'] }
data_comments=data_comments.map(text_concat)
data_comments

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text'],
    num_rows: 2175
})

In [16]:
print(data_comments['text'][0])

Protect master branch 
 After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:
- 00cc036fea7c7745cfe722360036ed306796a3f2
- 13ae8c98602bbad8197de3b9b425f4c78f582af1
- ...

I propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:
- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch
  - Currently, simple merge commits are already disabled
  - I propose to disable rebase merging as well
- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~
  - ~~This protection would reject direct pushes to master branch~~
  - ~~If so, for each release (when we need to commit directly to the master branch), we should previously disable the protection an

In [17]:
from transformers import AutoModel, AutoTokenizer
model_checkpoint="sentence-transformers/multi-qa-mpnet-base-dot-v1"
model=AutoModel.from_pretrained(model_checkpoint)
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [18]:
import torch
device=torch.device('cuda')
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [19]:
def cls_pooling(result):
  return result.last_hidden_state[:,0]

def get_embeddings(list_text):
  encoded_input=tokenizer(list_text,padding=True,truncation=True,return_tensors='pt')
  encoded_result={k: v.to(device) for k,v in encoded_input.items()}
  model_result=model(**encoded_result)
  return cls_pooling(model_result)

In [21]:
sample_embedding=get_embeddings(data_comments['text'][0])
sample_embedding.shape

torch.Size([1, 768])

In [22]:
embedding_data=data_comments.map(lambda x: {'embeddings':get_embeddings(x['text']).detach().cpu().numpy()[0]})
embedding_data

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [26]:
len(embedding_data['embeddings'])

2175

In [24]:
embedding_data.add_faiss_index(column='embeddings')
embedding_data

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [29]:
sample_question="HOw can I upload model on huggingface?"
question_embedding=get_embeddings(sample_question).detach().cpu().numpy()
question_embedding.shape

(1, 768)

In [31]:
scores,samples=embedding_data.get_nearest_examples('embeddings',question_embedding,k=5)

In [34]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [35]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

COMMENT: We already have this information in the dataset_infos.json files of each dataset.
Maybe we can parse these files in the backend to return their content with the endpoint at huggingface.co/api/datasets

For now if you want to access this info you have to load the json for each dataset. For example:
- for a dataset on github like `squad` 
-   https://raw.githubusercontent.com/huggingface/datasets/master/datasets/squad/dataset_infos.json
- for a community dataset on the hub like `lhoestq/squad`:
  https://huggingface.co/datasets/lhoestq/squad/resolve/main/dataset_infos.json
SCORE: 36.44215774536133
TITLE: add more precise information for size
URL: https://github.com/huggingface/datasets/issues/2714

COMMENT: This could be solved by going to the glue/ directory and delete sst2 directory, then load the dataset again will help you redownload the dataset.
SCORE: 36.06524658203125
TITLE: Missing cache file
URL: https://github.com/huggingface/datasets/issues/2722

COMMENT: Hi @yangp725